In [161]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from functools import reduce

## Max Plotitsyn PIW_11 v2

In [162]:
drugs = pd.read_csv('drugs.csv')
drugs.sort_values(by=['drug_category', 'drug_name'], inplace=True)
drugs

drug_id       drug_name drug_category
9       10        Firewall   Connecticut
1        2             Ftp   Connecticut
4        5  Representative   Connecticut
6        7           Agent      Keyboard
0        1           Metal      Keyboard
8        9      Overriding      Keyboard
3        4         Project      Keyboard
5        6           1080p      Navigate
2        3      Antarctica      Navigate
7        8     Compressing      Navigate

In [163]:
prices = pd.read_csv('price.csv')
prices

drug_id  drug_price  corporate_discount
0        1         366                  99
1        2         676                  66
2        3         560                  75
3        4          86                  38
4        5         153                  66
5        6          61                  24
6        7         398                  47
7        8         911                  35
8        9         959                  63
9       10         192                   6

In [164]:
sale1 = price = pd.read_csv('sales1.csv')
sale2 = price = pd.read_csv('sales2.csv')
sales = pd.concat([sale1, sale2], ignore_index=True).sort_values(by=['drug_id', 'corporate']).reset_index(drop=True)
sales = sales.groupby(['drug_id', 'corporate'], as_index=False).sum()
sales

drug_id corporate  drug_count
0         1        no          58
1         2        no          46
2         3        no         166
3         4       yes          29
4         5        no         113
5         6        no          78
6         6       yes          24
7         7       yes          39
8         8        no          45
9         8       yes          51
10        9       yes          52
11       10        no          58

In [165]:
big_df = reduce(lambda left,right: pd.merge(left, right, on='drug_id', how='left'), [drugs, prices, sales]).fillna(0)
big_df['drug_price'] = big_df['drug_price'].where(big_df['corporate'] == 'no', big_df['drug_price']-(big_df['drug_price']/100*big_df['corporate_discount'])).round(2)
big_df['saled'] = big_df['drug_price']*big_df['drug_count']
big_df

drug_id       drug_name drug_category  drug_price  corporate_discount  \
0        10        Firewall   Connecticut      192.00                   6   
1         2             Ftp   Connecticut      676.00                  66   
2         5  Representative   Connecticut      153.00                  66   
3         7           Agent      Keyboard      210.94                  47   
4         1           Metal      Keyboard      366.00                  99   
5         9      Overriding      Keyboard      354.83                  63   
6         4         Project      Keyboard       53.32                  38   
7         6           1080p      Navigate       61.00                  24   
8         6           1080p      Navigate       46.36                  24   
9         3      Antarctica      Navigate      560.00                  75   
10        8     Compressing      Navigate      911.00                  35   
11        8     Compressing      Navigate      592.15                  35   

   corporate  drug_count     saled  
0         no          58  11136.00  
1         no          46  31096.00  
2         no         113  17289.00  
3        yes          39   8226.66  
4         no          58  21228.00  
5        yes          52  18451.16  
6        yes          29   1546.28  
7         no          78   4758.00  
8        yes          24   1112.64  
9         no         166  92960.00  
10        no          45  40995.00  
11       yes          51  30199.65

## Task `1

In [166]:
def task1(category):
    print(category)
    bb = big_df.groupby(['drug_name', 'drug_category'], as_index=False).sum()
    print(bb[['drug_name', 'saled']][bb['drug_category'] == category].to_string(index=False))

interact(task1, category=sorted(list(set(big_df['drug_category']))));

interactive(children=(Dropdown(description='category', options=('Connecticut', 'Keyboard', 'Navigate'), value=…

## Task `2

In [167]:
def task2(category, plot_categories=False):
    bb = big_df.groupby(['drug_category'], as_index=False).sum().sort_values(by=['saled'], ascending=False)
    print(bb[['drug_category', 'saled']][bb['drug_category'] == category].to_string(index=False))
    if plot_categories:
        bb.plot(x='drug_category', y='saled', kind='bar', title='Saled', figsize=(10, 5), rot=30);

interact(task2, category=sorted(list(set(big_df['drug_category']))));

interactive(children=(Dropdown(description='category', options=('Connecticut', 'Keyboard', 'Navigate'), value=…

## Task `3

In [168]:
category = big_df.groupby(['drug_category', 'corporate'], as_index=False).sum().sort_values(by=['corporate', 'saled'], ascending=False).iloc[0]['drug_category']
big_df[['drug_name', 'saled']][(big_df['drug_category'] == category) & (big_df['corporate'] == 'yes')]

drug_name     saled
8         1080p   1112.64
11  Compressing  30199.65

## Task `4

In [169]:
big_df[big_df['drug_category'].isin(list(big_df['drug_category'][big_df['corporate'] == 'yes'].drop_duplicates()))].groupby(['drug_category'], as_index=False).sum().sort_values(by=['saled'], ascending=False)[['drug_category', 'saled']]

drug_category      saled
1      Navigate  170025.29
0      Keyboard   49452.10

## Task `5

In [170]:
big_df[['corporate', 'saled']][big_df['corporate'] == 'yes'].groupby('corporate', as_index=False).sum()

corporate     saled
0       yes  59536.39

## Task `6

In [172]:
def task3(category):
    print('drugs:')
    print(big_df['drug_name'][(big_df['drug_category'] == category) & (big_df['corporate'] == 'no')].to_string(index=False))
interact(task3, category=sorted(list(set(big_df['drug_category']))));

interactive(children=(Dropdown(description='category', options=('Connecticut', 'Keyboard', 'Navigate'), value=…